# Ratings prediction using SVD

We first try to gain some intuition on SVD using a small toy dataset. Let the following be a user-movie rating dataset where every user has rated every movie. 

In [4]:
import numpy as np
import pandas as pd

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

#df = pd.DataFrame([[1,2,8,9,3,3],[2,1,9,8,4,2],[2,2,6,8,2,3],
#                   [9,7,2,3,1,1],[1,1,1,2,8,7],[2,2,3,2,8,8],
#                   [7,9,2,2,2,3],[9,8,2,3,1,3]], 
#                  columns=["horror1","horror2","drama1","drama2","art1","art2"], 
#                  index=["u0","u1","u2","u3","u4","u5","u6","u7"])

df = pd.DataFrame([[1,2,8,9,3,3],[2,1,9,8,4,2],[2,2,6,8,2,3],
                   [9,7,2,3,1,1],[1,1,1,2,8,7],[2,2,3,2,8,8],
                   [7,9,2,2,2,3],[9,8,2,3,1,3],[7,1,1,9,2,8]], 
                  columns=["horror1","horror2","drama1","drama2","art1","art2"], 
                  index=["u0","u1","u2","u3","u4","u5","u6","u7","u8"])


df

,horror1,horror2,drama1,drama2,art1,art2
u0,1,2,8,9,3,3
u1,2,1,9,8,4,2
u2,2,2,6,8,2,3
u3,9,7,2,3,1,1
u4,1,1,1,2,8,7
u5,2,2,3,2,8,8
u6,7,9,2,2,2,3
u7,9,8,2,3,1,3
u8,7,1,1,9,2,8


Let us extract the data in the form of a matrix and subtract the mean from each row. 

In [5]:
A = df.values
means = np.mean(A,axis=1).reshape((A.shape[0],1))
#print(means)
A = A - means
print(A)

[[-3.33 -2.33 3.67 4.67 -1.33 -1.33]
 [-2.33 -3.33 4.67 3.67 -0.33 -2.33]
 [-1.83 -1.83 2.17 4.17 -1.83 -0.83]
 [5.17 3.17 -1.83 -0.83 -2.83 -2.83]
 [-2.33 -2.33 -2.33 -1.33 4.67 3.67]
 [-2.17 -2.17 -1.17 -2.17 3.83 3.83]
 [2.83 4.83 -2.17 -2.17 -2.17 -1.17]
 [4.67 3.67 -2.33 -1.33 -3.33 -1.33]
 [2.33 -3.67 -3.67 4.33 -2.67 3.33]]


Let us also examine on average what is the rating for each movie.

In [6]:
print(np.mean(A,axis=0))

[0.33 -0.44 -0.33 1.00 -0.67 0.11]


## Compute SVD

In [7]:
U, S, VT = np.linalg.svd(A, full_matrices=False)

print("U = \n", U, "\n")
print("S = ", S, "\n")
print("VT = \n", VT, "\n")

U = 
 [[-0.36 -0.40 -0.01 0.37 -0.18 0.70]
 [-0.37 -0.39 -0.13 -0.50 0.04 -0.21]
 [-0.24 -0.34 0.13 0.32 -0.15 -0.57]
 [0.45 -0.21 0.07 -0.49 -0.41 0.12]
 [-0.21 0.52 0.11 -0.03 -0.71 0.02]
 [-0.19 0.49 0.03 -0.03 0.43 0.07]
 [0.43 -0.03 -0.08 0.52 -0.15 -0.24]
 [0.46 -0.13 0.13 0.02 0.22 0.24]
 [-0.05 -0.06 0.96 -0.02 0.08 -0.01]] 

S =  [15.53 12.20 8.58 2.54 1.27 0.00] 

VT = 
 [[0.58 0.56 -0.35 -0.35 -0.28 -0.16]
 [-0.11 -0.04 -0.40 -0.52 0.56 0.50]
 [0.32 -0.39 -0.52 0.45 -0.30 0.43]
 [-0.61 0.57 -0.16 0.18 -0.34 0.36]
 [0.12 -0.19 0.51 -0.45 -0.49 0.50]
 [0.41 0.41 0.41 0.41 0.41 0.41]] 



## Dimension Reduction

We project the data onto the first $k$ singular vectors (optionally scaled by the corresponding singular values). This gives us the data in a reduced dimension, but should retain the most important information. 

In [9]:
k = 3
#Ak = np.diag(S[:k]) @ np.transpose(U[:,:k]) @ A
Ak = np.transpose(U[:,:k])  @ A
print(Ak)

[[8.96 8.76 -5.47 -5.43 -4.29 -2.52]
 [-1.32 -0.48 -4.84 -6.30 6.83 6.10]]


# Using some real data (MovieLens)

In [10]:
import numpy as np
import pandas as pd

ratings = pd.read_csv("/Users/debapriyo/Onedrive/data/ml-latest-small/ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## Separate some data for testing 

We will use this data for our experiments. But first, we need to separate some data so that we can test our methods on that. We create a random partition of the indices. 

In [34]:
# Create the list of indices
idx = np.arange(0,len(ratings))

# Randomly shuffle them
np.random.shuffle(idx)

# Size of test data
testsize = 5000
testidx = idx[0:testsize]
trainidx = idx[testsize:]
print(testidx)
print(trainidx)

[18743 33545 67099 ... 29237 66593  6859]
[46881 76302 66497 ... 17946 19210 10959]


Now, we partition the dataframe using the two sets of indices. 

In [35]:
# The sample for testing
ratings_test = ratings.iloc[testidx]
ratings_test

,userId,movieId,rating,timestamp
18743,119,141004,4.5,1505072481
33545,226,56174,4.0,1199115755
67099,434,293,4.5,1270603658
54758,362,431,4.0,1530638163
46011,305,1136,2.5,1463437043
...,...,...,...,...
72869,469,3398,3.0,965425863
8342,57,2108,3.0,965797154
29237,200,50442,2.0,1229877311
66593,428,4896,3.0,1111524229


In [36]:
# The rest of the data
ratings = ratings.iloc[trainidx]
ratings

,userId,movieId,rating,timestamp
46881,307,924,3.5,1186084299
76302,480,466,0.5,1179178797
66497,428,2428,2.0,1111524741
51997,338,183897,1.5,1530148463
56534,376,110,3.5,1364993995
...,...,...,...,...
32317,221,1080,4.5,1111178181
22028,144,1036,3.5,1137323713
17946,112,1198,5.0,1442535778
19210,124,2683,4.0,1336409883


## Creating the ratings matrix from the data

Instead of a 2-D array, we will create a sparse matrix first. We will use the rows as the items and columns as the users. However, we will have to make this matrix dense for some reasons. 

In [37]:
from scipy.sparse import csr_matrix

users = ratings["userId"].values.astype(int)
movies = ratings["movieId"].values.astype(int)
vals = ratings["rating"].values

RS = csr_matrix((vals,(users-1,movies-1)))

print("The data has %d items with %d dimensions (users)." %(RS.shape[1], RS.shape[0]))

The data has 193609 items with 610 dimensions (users).


In [38]:
# Converting it to a dense matrix
R = RS.todense()

print(R)

[[4.00 0.00 4.00 ... 0.00 0.00 0.00]
 [0.00 0.00 0.00 ... 0.00 0.00 0.00]
 [0.00 0.00 0.00 ... 0.00 0.00 0.00]
 ...
 [2.50 2.00 2.00 ... 0.00 0.00 0.00]
 [3.00 0.00 0.00 ... 0.00 0.00 0.00]
 [5.00 0.00 0.00 ... 0.00 0.00 0.00]]


## Fill the zero entries with average ratings 

The zero entries in this matrix are those for which the corresponding user has not rated the movie. In other words, those ratings are not zero, rather unknown. To start with, we fill the zero entries by the average ratings obtained by the movies (from users who have rated that movie). 

In [39]:
def colmeans(R):
    # Sum of the ratings for each movie
    sums = np.sum(R, axis=0)
    #print(sums.shape)

    # Number of non-zero entries for each movie (number of ratings)
    nzcounts = np.sum(R.astype(bool), axis=0) + 1
    #print(nzcounts.shape)

    # The average ratings for each movie
    means = sums/nzcounts
    #print(means.shape)
    
    return means

In [40]:
means = colmeans(R)
print(means.shape)

count = 0
for i in range(means.shape[1]):
    if (means[0,i]==0):
        count = count + 1
print(count)

(1, 193609)
184076


### Adding the mean vector (mean rating of each movie) to each column

Instead of running through loops, we will use masking. We create a Boolean matrix zero_R such that an entry of zero_R is True if and only if the corresponding entry of R is zero. 

In [41]:
# Demo on masking

M = np.array([[1,2,3,4],[0,1,0,2]])
print(M)
print(M.shape)

zero_M = M==0
print(zero_M)

# Ref. testing for RMSE_code 
#M_selected = M[[0,1],[2,3]]
#print(M_selected)

[[1 2 3 4]
 [0 1 0 2]]
(2, 4)
[[False False False False]
 [ True False  True False]]


In [42]:
# The boolean matrix (True/False entries) w

zero_R = R==0
print(zero_R.shape)
print(zero_R)

(610, 193609)
[[False  True False ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [False False False ...  True  True  True]
 [False  True  True ...  True  True  True]
 [False  True  True ...  True  True  True]]


Next, we multiply (element wise) zero_R to the broadcasted mean vector to get a matrix where an entry is zero if the corresponding entry in R was non zero, and an entry is the mean rating if the corresponding entry in R was zero (not rated).

In [43]:
unrated_R = np.multiply(means, zero_R)
print(unrated_R.shape)
print(unrated_R)

(610, 193609)
[[0.00 3.40 0.00 ... 0.00 0.00 2.00]
 [3.90 3.40 3.24 ... 0.00 0.00 2.00]
 [3.90 3.40 3.24 ... 0.00 0.00 2.00]
 ...
 [0.00 0.00 0.00 ... 0.00 0.00 2.00]
 [0.00 3.40 3.24 ... 0.00 0.00 2.00]
 [0.00 3.40 3.24 ... 0.00 0.00 2.00]]


Now, we add the unrated part of R with original R. 

In [44]:
R1 = R + unrated_R

print(R1)

[[4.00 3.40 4.00 ... 0.00 0.00 2.00]
 [3.90 3.40 3.24 ... 0.00 0.00 2.00]
 [3.90 3.40 3.24 ... 0.00 0.00 2.00]
 ...
 [2.50 2.00 2.00 ... 0.00 0.00 2.00]
 [3.00 3.40 3.24 ... 0.00 0.00 2.00]
 [5.00 3.40 3.24 ... 0.00 0.00 2.00]]


## A naive prediction of ratings by average ratings

The matrix R1 is already a naive prediction of the ratings. Let us test how good that is. First, we convert our test data (ratings which are known but were separated in the beginning) into a sparse matrix.

In [45]:
users_test = ratings_test["userId"].values.astype(int)
movies_test = ratings_test["movieId"].values.astype(int)
vals_test = ratings_test["rating"].values

print(users_test)
print(movies_test)
print(vals_test)

[119 226 434 ... 200 428  45]
[141004  56174    293 ...  50442   4896   8528]
[4.50 4.00 4.50 ... 2.00 3.00 5.00]


We write a function to compute Root Mean Squared Error (RMSE) for any matrix with same dimensions as R against the test data.

In [53]:
def RMSE_ratings(R_pred, users_test, movies_test, vals_test):
    R_pred_selected = R_pred[users_test-1,movies_test-1]
    #print(R_pred_selected.shape)
    error = R_pred_selected - vals_test
    #print(R_pred_selected)
    #print(vals_test)
    #print(error)
    return np.sqrt(np.mean(np.multiply(error,error)))

And we test the RMSE with the matrix R1.

In [47]:
print(RMSE_ratings(R1, users_test, movies_test, vals_test))

[[1.00 3.39 3.99 ... 2.88 3.74 3.42]]
[4.50 4.00 4.50 ... 2.00 3.00 5.00]
1.2187226604170796


## Back to SVD: Center the data (R1)

We center the data by making the mean of all columns zero. 

In [48]:
def centerrows(R1):
    # Center the columns (users)
    means = np.mean(R1, axis=1)
    #print(means)
    print(R1.shape)
    print(means.shape)
    return R1 - means

In [49]:
R2 = centerrows(R1)
print(R2)

(610, 193609)
(610, 1)
[[3.88 3.29 3.88 ... -0.12 -0.12 1.88]
 [3.78 3.29 3.12 ... -0.12 -0.12 1.88]
 [3.78 3.29 3.12 ... -0.12 -0.12 1.88]
 ...
 [2.38 1.88 1.88 ... -0.12 -0.12 1.88]
 [2.88 3.29 3.12 ... -0.12 -0.12 1.88]
 [4.88 3.28 3.12 ... -0.12 -0.12 1.88]]


## SVD 

Let us compute the SVD of the matrix now. 

In [50]:
U, S, VT = np.linalg.svd(R2, full_matrices=False)

print(U.shape, S.shape, VT.shape)

(610, 610) (610,) (610, 193609)


# The low rank approximation

The low rank approximation is defined by $R_k = U_k S_k V_k^T$. The dimensions will remain the same as before. 

In [51]:
def lowrank(U,S,VT,k):
    # Let us fix k

    Uk = U[:,:k]
    #print(Uk.shape)

    Sk = np.diag(S[:k])
    #print(Sk.shape)

    Vk = VT[:k,:]
    #print(Vk.shape)

    return Uk @ Sk @ Vk

## Moving back the columns from the center

Now we need to add the mean of the columns back.

In [54]:
k_range = np.array([5,10,20,30,50])
rmse = np.zeros(len(k_range))

for k in k_range:
    Rk1 = lowrank(U,S,VT,k) + R1 - R2
    #print(Rk1.shape)
    #print(Rk1[381,256])
    rmse = RMSE_ratings(Rk1, users_test, movies_test, vals_test)
    print("k = ", k, ", RMSE: ", rmse)

k =  5 , RMSE:  1.197982934748688
k =  10 , RMSE:  1.193518417676437
k =  20 , RMSE:  1.1947973621858419
k =  30 , RMSE:  1.1937472037132963
k =  50 , RMSE:  1.1959170215363482
